List Instances

In [6]:
import boto3
import os

# Initialize EC2 client
ec2_client = boto3.client("ec2", region_name="us-east-1")

# Fetch running instances
response = ec2_client.describe_instances(
    Filters=[
        {"Name": "instance-state-name", "Values": ["running"]}
    ]
)

# Function to check if an instance is unnamed
def is_unnamed_instance(instance):
    tags = instance.get("Tags", [])
    name_tag = next((tag for tag in tags if tag["Key"] == "Name"), None)
    return name_tag is None

# Collect unnamed Windows instances
unnamed_instances = []
for reservation in response['Reservations']:
    for instance in reservation["Instances"]:
        if "Windows" in instance.get("PlatformDetails", "") and is_unnamed_instance(instance):
            unnamed_instances.append({
                "InstanceId": instance["InstanceId"],
                "PublicDnsName": instance.get("PublicDnsName", ""),
                "PrivateIpAddress": instance.get("PrivateIpAddress", ""),
            })

# Output the results
if not unnamed_instances:
    print("No unnamed running instances")
else:
    print("Unnamed instances:")
    for inst in unnamed_instances:
        print(inst)


Unnamed instances:
{'InstanceId': 'i-0038533722f37d44f', 'PublicDnsName': 'ec2-54-211-209-116.compute-1.amazonaws.com', 'PrivateIpAddress': '172.31.45.202'}


List named instances

In [5]:
import boto3
import os

ec2_client = boto3.client("ec2", region_name="us-east-1")
response = ec2_client.describe_instances(
    Filters=[
        {"Name":"instance-state-name","Values":["running"]}
    ]
)
instances = []
for reservation in response['Reservations']:
    for instance in reservation["Instances"]:
        if "Windows" in instance.get("PlatformDetails",""):
            instances.append({
                "InstanceId":instance["InstanceId"],
                "PublicDnsName":instance.get("PublicDnsName",""),
                "PrivateIdAddress":instance.get("PrivateIpAddress",""),
            })

if not instances: 
    print("No running instance")
    exit()

print(instances)

[{'InstanceId': 'i-0569a45c521860ab1', 'PublicDnsName': 'ec2-54-152-112-179.compute-1.amazonaws.com', 'PrivateIdAddress': '172.31.36.240'}, {'InstanceId': 'i-0e143e1d115a5bc5e', 'PublicDnsName': 'ec2-100-24-15-229.compute-1.amazonaws.com', 'PrivateIdAddress': '172.31.40.178'}]


rename the unnamed instances

In [21]:
import boto3

# Function to get instances without a 'Name' tag
def get_unnamed_instances(instances):
    unnamed_instances = []
    for instance in instances:
        instance_id = instance['InstanceId']
        tags = instance.get('Tags', [])
        
        # Check if the 'Name' tag exists
        name_tag = next((tag for tag in tags if tag['Key'] == 'Name'), None)
        if not name_tag:
            unnamed_instances.append(instance)
    return unnamed_instances

# Function to rename unnamed EC2 instances
def rename_unnamed_ec2_instances(ec2_client, instances):
    unnamed_instances = get_unnamed_instances(instances)
    
    for i, instance in enumerate(unnamed_instances, start=1):
        instance_id = instance['InstanceId']
        new_name = f"Instance{i}"
        try:
            # Modify tags to set the new Name
            ec2_client.create_tags(
                Resources=[instance_id],
                Tags=[
                    {'Key': 'Name', 'Value': new_name}
                ]
            )
            print(f"Changed name of instance {instance_id} to {new_name}")
        except Exception as e:
            print(f"Failed to rename instance {instance_id}: {e}")

# Example usage
def main():
    ec2_client = boto3.client('ec2')  # Assuming boto3 is already imported
    try:
        response = ec2_client.describe_instances()
        all_instances = []
        
        # Extract all instances from all reservations
        for reservation in response['Reservations']:
            all_instances.extend(reservation['Instances'])
        
        # Rename unnamed EC2 instances
        rename_unnamed_ec2_instances(ec2_client, all_instances)
    except Exception as e:
        print(f"Failed to retrieve or process instances: {e}")

if __name__ == "__main__":
    main()


Export InstanceID, InstanceName in CSV (newly created instances)

In [33]:
import boto3
import csv
import re

# Initialize EC2 client
ec2_client = boto3.client("ec2", region_name="us-east-1")

# Fetch running instances
response = ec2_client.describe_instances(
    Filters=[{"Name": "instance-state-name", "Values": ["running"]}]
)

# Extract instance details
instances = []
for reservation in response.get('Reservations', []):
    for instance in reservation.get("Instances", []):
        tags = {tag['Key']: tag['Value'] for tag in instance.get("Tags", [])}
        name = tags.get("Name", "Unnamed")  # Default to 'Unnamed' if no Name tag
        instances.append({
            "InstanceId": instance["InstanceId"],
            "Name": name
        })

# Debug: Print all instances
print("Fetched Instances:")
for instance in instances:
    print(f"InstanceId: {instance['InstanceId']}, Name: {instance['Name']}")

# Filter instances matching the formats "Instance1", "Instance 1", "NewInstance1", or "NewInstance 1"
filtered_instances = [
    instance for instance in instances
    if re.match(r"^Instance\s?\d+$", instance['Name'], re.IGNORECASE) or re.match(r"^NewInstance\s?\d+$", instance['Name'], re.IGNORECASE)
]

# Debug: Print filtered instances
print("Filtered Instances:")
for instance in filtered_instances:
    print(f"InstanceId: {instance['InstanceId']}, Name: {instance['Name']}")

# Write to CSV
csv_file = 'filtered_instances.csv'
if filtered_instances:
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['InstanceId', 'InstanceName'])  # Write header
        for instance in filtered_instances:
            writer.writerow([instance['InstanceId'], instance['Name']])  # Write InstanceId and Name
    print(f"Filtered InstanceIds and Names have been written to {csv_file}.")
else:
    print("No instances matching the required naming formats found.")


Fetched Instances:
InstanceId: i-0fbb57b7d8d45da26, Name: st-app
InstanceId: i-0ff470748449c8b3b, Name: NewInstance1
InstanceId: i-070cf9a4bf3931e73, Name: NewInstance2
InstanceId: i-0c43f0e8f9d13ea86, Name: st-app2
InstanceId: i-09e8ea2044d237791, Name: Del1
InstanceId: i-07979fa7c6cc78c4c, Name: Del2
InstanceId: i-07569b18840d312c8, Name: Instance1
InstanceId: i-0d43625624b2e7209, Name: Instance2
Filtered Instances:
InstanceId: i-0ff470748449c8b3b, Name: NewInstance1
InstanceId: i-070cf9a4bf3931e73, Name: NewInstance2
InstanceId: i-07569b18840d312c8, Name: Instance1
InstanceId: i-0d43625624b2e7209, Name: Instance2
Filtered InstanceIds and Names have been written to filtered_instances.csv.


Export InstanceID, InstanceName in CSV

In [32]:
import csv

for instance in instances:
    print(instance['InstanceId'] + "  "+ instance['Name'])

# Define the CSV file name
csv_file = 'instance_ids.csv'

# Write InstanceId and Name to the CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Id', 'Name'])  # Writing the header
    for instance in instances:
        writer.writerow([instance['InstanceId'], instance['Name']])  # Writing each InstanceId and Name

print(f"InstanceIds and Names have been written to {csv_file}.")

KeyError: 'InstanceId'

Download rdp files of filtered instances

In [34]:
import boto3
import csv

# Initialize EC2 client
ec2_client = boto3.client("ec2", region_name="us-east-1")

# Read filtered instances from CSV file
filtered_instances = []
csv_file = 'filtered_instances.csv'

try:
    with open(csv_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            filtered_instances.append(row["InstanceId"])
except Exception as e:
    print(f"Error reading filtered instances from CSV: {str(e)}")

# Debug: Print filtered instances from CSV
print("Filtered Instance IDs from CSV:", filtered_instances)

# Fetch details for filtered instances
instances_to_rdp = []
response = ec2_client.describe_instances(
    InstanceIds=filtered_instances
)
for reservation in response.get('Reservations', []):
    for instance in reservation.get("Instances", []):
        public_dns = instance.get("PublicDnsName", "")
        name_tag = next((tag["Value"] for tag in instance.get("Tags", []) if tag["Key"] == "Name"), "Unnamed")
        if public_dns:  # Only add instances with a valid public DNS
            instances_to_rdp.append({
                "InstanceId": instance["InstanceId"],
                "Name": name_tag,
                "PublicDnsName": public_dns
            })

# Debug: Print instances with valid public DNS
print("Instances with valid Public DNS for RDP:")
for instance in instances_to_rdp:
    print(f"InstanceId: {instance['InstanceId']}, Name: {instance['Name']}, PublicDnsName: {instance['PublicDnsName']}")

# Write RDP files for instances with valid Public DNS
for instance in instances_to_rdp:
    instance_id = instance['InstanceId']
    instance_name = instance['Name']
    try:
        rdp_file_name = f"{instance_name}.rdp"
        with open(rdp_file_name, "w") as rdp_file:
            rdp_file.write(f"full address:s:{instance['PublicDnsName']}\n")
            rdp_file.write("username:s:Administrator\n")
        print(f'RDP file downloaded for instance: {instance_name} -> {rdp_file_name}')
    except Exception as e:
        print(f"Error downloading RDP file for {instance_name}: {str(e)}")


Filtered Instance IDs from CSV: ['i-0ff470748449c8b3b', 'i-070cf9a4bf3931e73', 'i-07569b18840d312c8', 'i-0d43625624b2e7209']
Instances with valid Public DNS for RDP:
InstanceId: i-0ff470748449c8b3b, Name: NewInstance1, PublicDnsName: ec2-54-226-181-45.compute-1.amazonaws.com
InstanceId: i-070cf9a4bf3931e73, Name: NewInstance2, PublicDnsName: ec2-54-204-174-187.compute-1.amazonaws.com
InstanceId: i-07569b18840d312c8, Name: Instance1, PublicDnsName: ec2-50-16-120-229.compute-1.amazonaws.com
InstanceId: i-0d43625624b2e7209, Name: Instance2, PublicDnsName: ec2-54-224-136-237.compute-1.amazonaws.com
RDP file downloaded for instance: NewInstance1 -> NewInstance1.rdp
RDP file downloaded for instance: NewInstance2 -> NewInstance2.rdp
RDP file downloaded for instance: Instance1 -> Instance1.rdp
RDP file downloaded for instance: Instance2 -> Instance2.rdp


RDP path in attachment.csv

In [36]:
import pandas as pd
import os
# Define the folder path
folder_path = 'c:\\Hackathon\\rdp'  # Replace with the path to your folder
output_csv = 'attachments.csv'     # Output CSV file name
 
# Get the list of all files in the folder
file_paths = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_paths.append(os.path.join(root, file))
 
# Create a DataFrame to store the file paths
df = pd.DataFrame(file_paths, columns=['Attachments'])
 
# Save the file paths to a CSV file
df.to_csv(output_csv, index=False)
 
print(f"File paths have been saved to {output_csv}.")

File paths have been saved to attachments.csv.


mainsheet(email, instanceID, instance name)

In [38]:
import csv

# File paths
participants_file = 'participants1.csv'
filtered_instances_file = 'filtered_instances.csv'
output_file = 'mainsheet.csv'

# Read Email from Participants1.csv
emails = []
try:
    with open(participants_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            emails.append(row['Email'])  # Assuming 'Email' is the column name
except Exception as e:
    print(f"Error reading {participants_file}: {str(e)}")

# Debug: Print emails
print("Emails from participants1.csv:", emails)

# Read InstanceID and InstanceName from filtered_instances.csv
instances = []
try:
    with open(filtered_instances_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            instances.append({
                'InstanceID': row['InstanceId'],  # Assuming 'InstanceId' is the column name
                'InstanceName': row['InstanceName']  # Assuming 'InstanceName' is the column name
            })
except Exception as e:
    print(f"Error reading {filtered_instances_file}: {str(e)}")

# Debug: Print instances
print("Instances from filtered_instances.csv:", instances)

# Combine data into mainsheet.csv
try:
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Email', 'InstanceID', 'InstanceName'])  # Write header

        # Match emails to instances (1-to-1 matching)
        for email, instance in zip(emails, instances):
            writer.writerow([email, instance['InstanceID'], instance['InstanceName']])

    print(f"Mainsheet created successfully: {output_file}")
except Exception as e:
    print(f"Error writing {output_file}: {str(e)}")


Emails from participants1.csv: ['gsingla@deloitte.com', 'tipanchal@deloitte.com', 'lnair@deloitte.com', 'ankamboj@deloitte.com', 'modhamor@deloitte.com', 'ngupta14@deloitte.com', 'hsoni@deloitte.com', 'kalisinghal@deloitte.com', 'shriysingh@deloitte.com', 'achadda@deloitte.com', 'prjawahar@deloitte.com', 'prapunjabi@deloitte.com', 'nakrathore@deloitte.com', 'posomani@deloitte.com', 'smjoshi@deloitte.com', 'kushgupta@deloitte.com', 'gaghag@deloitte.com', 'ppatwa@deloitte.com', 'maloke@deloitte.com', 'gets1@deloitte.com', 'mansdang@deloitte.com', 'vshah11@deloitte.com', 'srasal@deloitte.com', 'prpal@deloitte.com', 'harinr@deloitte.com', 'achattarbandh@deloitte.com', 'atyagi7@deloitte.com', 'sebano@deloitte.com', 'Duggalm@deloitte.com', 'deumesh@deloitte.com', 'skhattak@deloitte.com', 'mejaiswal@deloitte.com', 'BHUMIKAJAIN.EXT@DELOITTE.COM', 'sonmahato@deloitte.com', 'sosubramanian@deloitte.com', 'pamukhtyar@deloitte.com', 'dburma@deloitte.com', 'arbehal@deloitte.com', 'vischavan@deloitte

Email file(Email_data.csv)

In [ ]:
import pandas as pd

# Load the CSV files
mainsheet_df = pd.read_csv('mainsheet.csv')  # Fetch Email from mainsheet.csv
attachments_df = pd.read_csv('attachments.csv')  # Fetch Attachments from attachments.csv

# Creating the subject and body as constants
subject = "Get Started: RDP File and Access Details for Galactic Hackathon 2025"
body = """Hi,

We are delighted to welcome you to the Galactic Hackathon 2025. Below, you will find your credentials to access the necessary resources and participate in the event:

Login Credentials:

Username: Administrator
Password: DelloiteHackathon@2025
Access File: Provided in attachments (Download and open the .rdp file where you are supposed to log in via provided credentials)

Please keep this information confidential and do not share it with others. If you encounter any issues accessing the platform or have questions, feel free to contact us.

We look forward to your active participation and wish you a wonderful experience!

Best regards,
The Knowledge Academy Team
"""

# Merge mainsheet (Emails) with attachments to ensure proper alignment of data
email_data_df = pd.DataFrame({
    'Email': mainsheet_df['Email'],  # Emails from mainsheet.csv
    'Subject': subject,
    'Body': body,
    'Attachment': attachments_df['Attachments']  # Attachments from attachments.csv
})

# Save the updated data to a different location (e.g., Desktop)
output_path = r'c:\\Hackathon\\email_data.csv'
email_data_df.to_csv(output_path, index=False, lineterminator='\n')

print(f"email_data.csv has been updated and saved to {output_path}.")


email_data.csv has been updated and saved to c:\\Hackathon\\email_data.csv.


zoom meeting data

Sending Emails

In [37]:
import os
import pandas as pd
import win32com.client as win32

# Initialize Outlook
outlook = win32.Dispatch('Outlook.Application')

# Read email data from CSV with a specified encoding
try:
    data = pd.read_csv('email_data.csv', encoding='latin1')  # Use 'latin1' or other encodings as needed
except UnicodeDecodeError as e:
    print(f"Failed to read CSV file: {e}")
    exit()

# Iterate through each row in the data
for index, row in data.iterrows():
    # Access the row data using .iloc to avoid deprecation warnings
    recipient = row.iloc[0]  # First column: Email
    subject = row.iloc[1]    # Second column: Subject
    body = row.iloc[2]       # Third column: Body
    attachment_path = row.iloc[3]  # Fourth column: Attachment

    # Create an email item
    try:
        mail = outlook.CreateItem(0)  # 0 represents an email item
        mail.To = recipient
        mail.Subject = subject
        mail.Body = body

        # Attach the file if it exists
        if os.path.exists(attachment_path):
            try:
                mail.Attachments.Add(attachment_path)  # Add attachment if file exists
            except Exception as e:
                print(f"Failed to attach {attachment_path}: {e}")
        else:
            print(f"Attachment file not found: {attachment_path}")

        # Send the email
        try:
            mail.Send()
            print(f"Email sent to {recipient}")
        except Exception as e:
            print(f"Failed to send email to {recipient}: {e}")
    except Exception as e:
        print(f"Failed to create email for {recipient}: {e}")


Email sent to jai.prakash@theknowledgeacademy.com
Email sent to spandana.ts@theknowledgeacademy.com
Email sent to ashutosh.kushwaha@theknowledgeacademy.com
Email sent to vinita.sinha@theknowledgeacademy.com


Sheets for output.csv

In [ ]:


import csv

# Define the file names
file1 = 'file1.csv'  # File with InstanceId, Name, Email
file2 = 'file2.csv'  # File with Email only
output_file = 'output.csv'  # File to save the results

# Read the first file into a dictionary with email as the key
file1_data = {}
with open(file1, mode='r', newline='', encoding='utf-8') as f1:
    reader = csv.DictReader(f1)
    for row in reader:
        file1_data[row['Email']] = {'InstanceId': row['InstanceId'], 'InstanceName': row['InstanceName']}

# Find the matching rows based on email from the second file
matching_data = []
with open(file2, mode='r', newline='', encoding='utf-8') as f2:
    reader = csv.DictReader(f2)
    for row in reader:
        email = row['Email']
        if email in file1_data:
            matching_data.append({
                'InstanceId': file1_data[email]['InstanceId'],
                'InstanceName': file1_data[email]['InstanceName'],
                'Email': email
            })

# Write the matching data to an output file
with open(output_file, mode='w', newline='', encoding='utf-8') as f_out:
    writer = csv.DictWriter(f_out, fieldnames=['InstanceId', 'InstanceName', 'Email'])
    writer.writeheader()
    writer.writerows(matching_data)

print(f"Matching rows have been written to {output_file}.")


Absentees

In [ ]:
import csv

# Define the file names
file1 = 'file1.csv'  # File with InstanceId, Name, Email
file2 = 'file2.csv'  # File with Email only
output_file = 'output.csv'  # File to save the matching results
absentees_file = 'absentees.csv'  # File to save the absentees

# Read the first file into a dictionary with email as the key
file1_data = {}
with open(file1, mode='r', newline='', encoding='utf-8') as f1:
    reader = csv.DictReader(f1)
    for row in reader:
        file1_data[row['Email']] = {'InstanceId': row['InstanceId'], 'InstanceName': row['InstanceName']}

# Create a set of emails from file2.csv
file2_emails = set()
with open(file2, mode='r', newline='', encoding='utf-8') as f2:
    reader = csv.DictReader(f2)
    for row in reader:
        file2_emails.add(row['Email'])

# Find matching data
matching_data = []
absentees_data = []

for email, data in file1_data.items():
    if email in file2_emails:
        matching_data.append({'InstanceId': data['InstanceId'], 'InstanceName': data['InstanceName'], 'Email': email})
    else:
        absentees_data.append({'InstanceId': data['InstanceId'], 'InstanceName': data['InstanceName'], 'Email': email})

# Write the matching data to an output file
with open(output_file, mode='w', newline='', encoding='utf-8') as f_out:
    writer = csv.DictWriter(f_out, fieldnames=['InstanceId', 'InstanceName', 'Email'])
    writer.writeheader()
    writer.writerows(matching_data)

# Write the absentees data to a separate file
with open(absentees_file, mode='w', newline='', encoding='utf-8') as f_absent:
    writer = csv.DictWriter(f_absent, fieldnames=['InstanceId', 'InstanceName', 'Email'])
    writer.writeheader()
    writer.writerows(absentees_data)

print(f"Matching rows have been written to {output_file}.")
print(f"Absentees have been written to {absentees_file}.")


Stop Instances

In [ ]:
import boto3
import csv

def stop_instances_from_csv(file_path, region_name='us-east-1'):
    ec2_client = boto3.client('ec2', region_name=region_name)

    try:
        # Read instance IDs from the CSV file
        instance_ids = []
        with open(file_path, mode='r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader)  # Skip the header row if present
            for row in csv_reader:
                if row:
                    instance_ids.append(row[0])  # Assuming instance ID is in the first column

        if not instance_ids:
            print("No instance IDs found in the CSV file.")
            return

        print(f"Instance IDs to stop: {instance_ids}")

        # Stop the instances
        response = ec2_client.stop_instances(InstanceIds=instance_ids)
        stopped_instances = response.get('StoppingInstances', [])

        for instance in stopped_instances:
            print(f"Stopping instance {instance['InstanceId']}: Current state - {instance['CurrentState']['Name']}, Previous state - {instance['PreviousState']['Name']}")

        print("Instances stopped successfully.")

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    csv_file_path = "absentees.csv"  # Path to your CSV file
    print("Stopping instances listed in the CSV file...")
    stop_instances_from_csv(csv_file_path)